# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235457038863                   -0.50    8.0
  2   -7.250428347548       -1.82       -1.40    1.0
  3   -7.251315963184       -3.05       -2.15    4.0
  4   -7.251287766004   +   -4.55       -2.28    3.0
  5   -7.251328440339       -4.39       -2.66    2.0
  6   -7.251337703051       -5.03       -3.10    2.0
  7   -7.251338760777       -5.98       -3.92    1.0
  8   -7.251338791390       -7.51       -4.10    4.0
  9   -7.251338797672       -8.20       -4.52    2.0
 10   -7.251338798529       -9.07       -4.95    2.0
 11   -7.251338798702       -9.76       -5.66    3.0
 12   -7.251338798704      -11.55       -6.30    3.0
 13   -7.251338798704      -13.32       -6.54    3.0
 14   -7.251338798705      -13.03       -6.95    3.0
 15   -7.251338798705      -13.85       -7.89    3.0
 16   -7.251338798705   +  -14.45       -8.18    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08237171530050631
[ Info: Arnoldi iteration step 2: normres = 0.5321243569017696
[ Info: Arnoldi iteration step 3: normres = 0.7231010933079147
[ Info: Arnoldi iteration step 4: normres = 0.2520296842556132
[ Info: Arnoldi iteration step 5: normres = 0.6178606899080716
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.84e-02, 9.88e-02, 5.82e-01, 1.79e-01, 1.16e-02)
[ Info: Arnoldi iteration step 6: normres = 0.22157825994448085
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.51e-03, 1.00e-01, 1.14e-01, 1.22e-01, 7.06e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07405818415139545
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.58e-04, 6.47e-03, 7.32e-03, 4.86e-02, 5.01e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08890854013733271
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.16e-